In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

#initialising  prepare_callbacks stuff
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboad_root_log_dir: Path
    checkpoint_model_filepath: Path


In [ ]:
from classifier.constants import *
from classifier.utils.common_functions import read_yaml, create_dirctories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            parameters_filepath= PARAMETERS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.parameters= read_yaml(parameters_filepath)
        create_dirctories([self.config.artifacts_roots])
                

    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpont_model_filepath)

        create_dirctories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [ ]:
import os
import urllib.request as requests
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        time_stamp= time.strftime("%Y-%M-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{time_stamp}",

        )

        


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath,
            save_best_only= True
        )
    


    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [ ]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callbacks_config
    prepare_callbacks= PrepareCallbacks(config=prepare_callbacks_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

In [ ]:
import time
timestamp= time.strftime("%Y-%M-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"